In [1]:
# 분석용 서버 구축을 위한 flask 설치
!pip install flask

In [15]:
!pip install matplotlib

In [6]:
# 영상처리분야에서  많이 쓰는 라이브러리
!pip install opencv-python

In [7]:
from flask import Flask, request, Response, redirect
import io # 파이썬의 입출력을 도와주는 라이브러리
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model # 딥러닝 모델 로딩 함수
import cv2 # opencv

In [16]:
# 플라스크 객체 생성
app = Flask(__name__)
# 학습된 모델 로딩
model = load_model('animal_keras_model.h5')

# 사용자 요청을 처리하기 위한 라우터 설정
@app.route('/', methods=['GET','POST'])
def index():
    return redirect("http://localhost:8081/Test/result.jsp")

@app.route('/predict', methods=['GET','POST'])
def predict():
    if request.method == 'POST':
        img = request.files['uploadImage']
        input_stream = io.BytesIO() #byte단위로 읽어들이는 통로
        img.save(input_stream) # 파일로부터 데이터를 읽기
        data = np.fromstring(input_stream.getvalue(),
                            dtype=np.uint8)
        real_img = cv2.imdecode(data,1) # 1 => 컬러사진
        real_img = cv2.cvtColor(real_img,cv2.COLOR_BGR2RGB) # BGR에서 RGB
        # 이미지 크기 조절하기
        resize_img = cv2.resize(real_img, dsize=(224,224), interpolation=cv2.INTER_AREA)
        
        # 이미지 확인하기
        #display(plt.imshow(resize_img))
        #plt.show()
        
        # 이미지 정규화(-1 ~ 1)
        normalized_img = (np.array(resize_img,dtype=np.float32) / 127.0) - 1
        
        # 예측시킬 이미지 데이터를 넣을 변수
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        
        # 불러온 numpy 타입의 이미지를 변수에 대입
        data[0] = normalized_img
        
        # 예측
        prediction = model.predict(data)
        index = np.argmax(prediction[0]) # 최대값의 인덱스
        
        
    return redirect("http://localhost:8081/Test/result.jsp?pre={}".format(index))

app.run() #서버 구동

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
C:\Users\smhrd\anaconda3\envs\animal_deep\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
127.0.0.1 - - [19/Apr/2022 10:58:00] "POST /predict HTTP/1.1" 302 -
